In [5]:
import os
import mlflow
from dagshub import dagshub_logger
from bertopic import BERTopic

In [2]:
from src.process_data import read_yaml
from src.topic_model import train_bert, load_bert, visualize_topics

/home/eugenia/topic-modeling-reviews/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
params = read_yaml()

In [7]:
topic_model = load_bert(params['model_path'])

ModuleNotFoundError: No module named 'topic_model'

In [4]:
if params['visualization_over']==False:
    exec(open("src/visualization.py").read())

ImportError: attempted relative import with no known parent package

In [7]:
topic_model = load_bert(params['model_path'])
print(topic_model.get_topic_freq().head())

ModuleNotFoundError: No module named 'topic_model'

In [4]:
topic_model = load_bert(params['model_path'])
print(topic_model.get_topic_freq().head())
print('Create visualizations!')
visualize_topics(topic_model,docs)

ModuleNotFoundError: No module named 'process_data'